# Lab 4: Model Context Protocol (MCP) as Tools

Welcome to Lab 4 of our Strands Agent course! So far, we've created basic agents and connected them with different model providers.  Now we will empower these agents with tools that will allow them to interact with external systems, access data, and manipulate their environment. And you're going to learn how to use the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) as one of those tools.



### Let's create a Weather Agent

Strands agents offer an [Example Tools Package](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/tools_overview/#adding-tools-to-agents). You can also create [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) similar to MCP implementation, but without the need to run a dedicated server.

To create the weather agent, we use the [http_request tool](https://github.com/strands-agents/tools/blob/main/src/strands_tools/http_request.py). This tool connects Strands agents with external web services and APIs, bridging conversational AI with data sources. The tool supports multiple HTTP methods (GET, POST, PUT, DELETE), handles URL encoding and response parsing, and returns structured data from web sources.

![weather_agent](image/weather_agent.png)

In [ ]:

!pip install -r requirements.txt

In [ ]:
api_key="your-api-key"

In [ ]:
from strands import Agent
from strands_tools import http_request
from strands.models.anthropic import AnthropicModel
import os

# Define a weather-focused system prompt
WEATHER_SYSTEM_PROMPT = """You are a weather assistant with HTTP capabilities. You can:

1. Make HTTP requests to the National Weather Service API
2. Process and display weather forecast data
3. Provide weather information for locations in the United States

When retrieving weather information:
1. First get the coordinates or grid information using https://api.weather.gov/points/{latitude},{longitude} or https://api.weather.gov/points/{zipcode}
2. Then use the returned forecast URL to get the actual forecast

When displaying responses:
- Format weather data in a human-readable way
- Highlight important information like temperature, precipitation, and alerts
- Handle errors appropriately
- Convert technical terms to user-friendly language

Always explain the weather conditions clearly and provide context for the forecast.
"""

model = AnthropicModel(
    client_args={
         "api_key": api_key #os.getenv("api_key") #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

# Create an agent with HTTP capabilities
weather_agent = Agent(
    model = model,
    system_prompt=WEATHER_SYSTEM_PROMPT,
    tools=[http_request],  # Explicitly enable http_request tool
)

### ✅ Test the Weather agent

In [ ]:
response = weather_agent("What's the weather like in Seattle?")


In [ ]:
print(response.message['content'][0]['text'])

### ✅  Weather agent with a custom tool (Python Tool)

![weather_agent_tool](image/weather_agent_tool.png)

In [ ]:
from strands import Agent
from strands_tools import http_request
from strands.models.anthropic import AnthropicModel
from strands.tools import tool
from typing import Dict, Any

# Define a function to convert Fahrenheit to Celsius
@tool
def fahrenheit_to_celsius(params: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert temperature from Fahrenheit to Celsius.

    Args:
        params: A dictionary containing:
            - fahrenheit: The temperature in Fahrenheit to convert

    Returns:
        A dictionary containing:
            - celsius: The temperature in Celsius
            - original_fahrenheit: The original Fahrenheit value
    """
    try:
        fahrenheit = float(params.get("fahrenheit"))
        celsius = (fahrenheit - 32) * 5/9
        return {
            "celsius": round(celsius, 2),
            "original_fahrenheit": fahrenheit
        }
    except (TypeError, ValueError):
        return {
            "error": "Invalid input. Please provide a valid number for fahrenheit."
        }


# Define a weather-focused system prompt
WEATHER_SYSTEM_PROMPT_C_TOOL = """You are a weather assistant with HTTP capabilities. You can:

1. Make HTTP requests to the National Weather Service API
2. Process and display weather forecast data
3. Provide weather information for locations in the United States
4. Use fahrenheit_to_celsius to convert temperatures between Fahrenheit and Celsius

When retrieving weather information:
1. First get the coordinates or grid information using https://api.weather.gov/points/{latitude},{longitude} or https://api.weather.gov/points/{zipcode}
2. Then use the returned forecast URL to get the actual forecast

When displaying responses:
- Format weather data in a human-readable way
- Highlight important information like temperature, precipitation, and alerts
- Handle errors appropriately
- Convert technical terms to user-friendly language

Always explain the weather conditions clearly and provide context for the forecast.
"""

model = AnthropicModel(
    client_args={
        "api_key": api_key, #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

# Create an agent with HTTP capabilities
weather_agent_c_tool = Agent(
    model = model,
    system_prompt=WEATHER_SYSTEM_PROMPT_C_TOOL,
    tools=[http_request,fahrenheit_to_celsius],  # Explicitly enable http_request tool
)

In [ ]:
print(weather_agent_c_tool.tool_names)

In [ ]:
response_c_tool = weather_agent_c_tool("What is the weather like in Seattle? Please give me the answer in degrees Celsius")


In [ ]:
print(response_c_tool.message['content'][0]['text'])

## The Model Context Protocol (MCP)

The Model Context Protocol (MCP) is an open protocol that standardized way to expose and consume tools across different systems. 

This approach is ideal for creating reusable tool collections that can be shared across multiple agents or applications. 

You can create your own MCP server to extend agent capabilities. 

![mcp](image/mcp.png)

### MCP Architecture Components

- **MCP Client**: Integrated in Strands Agent Framework 

- **MCP Server**: Independent service hosting tools

- **MCP Tools**: Specific functionalities (calculator, weather, etc.)


## Let's create a Calculator MCP server.

In this section, we will do a walkthrough of selected code blocks, that have been used to build [mcp_calculator.py](mcp_calulator.py) to create a simple MCP server that provides limited calculator functionality.

![calculator](image/calculator.png)


### ✅ Create the MCP server 


```python
from mcp.server import FastMCP

mcp = FastMCP("Calculator Server")

@mcp.tool(description="Add two numbers together")
def add(x: int, y: int) -> int:
"""Add two numbers and return the result."""
    return x + y

# Define a subtraction tool
@mcp.tool(description="Subtract one number from another")
def subtract(x: int, y: int) -> int:
        """Subtract y from x and return the result.

        Args:
            x: Number to subtract from
            y: Number to subtract

        Returns:
            The difference (x - y)
        """
    return x - y

# Define a multiplication tool
@mcp.tool(description="Multiply two numbers together")
def multiply(x: int, y: int) -> int:
        """Multiply two numbers and return the result.

        Args:
            x: First number
            y: Second number

        Returns:
            The product of x and y
        """
    return x * y

# Define a division tool
@mcp.tool(description="Divide one number by another")
def divide(x: float, y: float) -> float:
        """Divide x by y and return the result.

        Args:
            x: Numerator
            y: Denominator (must not be zero)

        Returns:
            The quotient (x / y)

        Raises:
            ValueError: If y is zero
        """
    if y == 0:
        raise ValueError("Cannot divide by zero")
    return x / y
```

[mcp_calculator.py](mcp_calulator.py) uses Streamable HTTP as MCP Server Connection 

```python
mcp.run(transport="streamable-http")
```

### ✅ Connecting the server to the Strands Agent

The following code shows how to create a Strands agent, and connect it to a MCP server:

```python
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient
from strands.models.anthropic import AnthropicModel

model = AnthropicModel(
    client_args={
        "api_key": "<KEY>", #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)
    # Create a system prompt that explains the calculator capabilities
    system_prompt = """
    You are a helpful calculator assistant that can perform basic arithmetic operations.
    You have access to the following calculator tools:
    - add: Add two numbers together
    - subtract: Subtract one number from another
    - multiply: Multiply two numbers together
    - divide: Divide one number by another
    
    When asked to perform calculations, use the appropriate tool rather than calculating the result yourself.
    Explain the calculation and show the result clearly.
    """

def create_streamable_http_transport():
   return streamablehttp_client("http://localhost:8000/mcp/")

streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

# Use the MCP server in a context manager
with streamable_http_mcp_client:
    # Get the tools from the MCP server
    tools = streamable_http_mcp_client.list_tools_sync()
    
    # Create an agent with the MCP tools
    agent = Agent(model=model,system_prompt=system_prompt,tools=tools)

```




### ✅ Test the calculator agent

Go to the terminal and run the app

```

python3 mcp_calulator.py

```

![calculator](image/calculator.gif)


